# Module 2: Mixed Complementarity Problems (MCPs) and Strategic Competition

In this module we inspect market-equilibrium problems expressed as **mixed complementarity problems (MCPs)**. The modeling exercises in this notebook include:

1)  Duopoly energy producer problem

2)  Energy network market equilibrium

#### Import code packages

In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.mpec import Complementarity, complements
import os

In [ ]:
# Set up the PATH solver
# Set the PATH license string (from https://pages.cs.wisc.edu/~ferris/path.html)
os.environ['PATH_LICENSE_STRING'] = "2830898829&Courtesy&&&USR&45321&5_1_2021&1000&PATH&GEN&31_12_2025&0_0_0&6000&0_0"

# Set path to PATH executable (dependent on your OS)
path_executable = os.path.join(os.getcwd(), 'path', 'pathampl')


!chmod +x /content/path/pathampl

## Exercise 2.1: Duopoly competition model

#### Problem Description

We analyze a **duopoly competition model** where two energy producers compete in quantities (Cournot competition). The problem is formulated as a **Linear Complementarity Problem (LCP)** derived from the first-order conditions of profit maximization.

#### Mathematical Formulation

**Complementarity Conditions:**

$0 \leq q_i \perp F(q_i) \geq 0$ for $i = 1,2$

where:

$F\begin{pmatrix} q_1 \\ q_2 \end{pmatrix} = \begin{pmatrix} 2\beta q_1 + \beta q_2 - \alpha + \gamma_1 \\ \beta q_1 + 2\beta q_2 - \alpha + \gamma_2 \end{pmatrix}$


**Parameters:**
- $\alpha = 10$ (market demand intercept)
- $\beta = 5$ (demand slope parameter)
- $\gamma_1 = 1$ (producer 1's cost parameter)
- $\gamma_2 = 1$ (producer 2's cost parameter)

Notice that this problem consists of **only complementarity conditions**, without an objective function. Hence, we need to use the **PATH** solver to solve it.

#### Define Problem Data

In [ ]:
# Cost Parameter
gamma1 = 1
gamma2 = 1

# Demand function parameters
a = 10
b = 5

### Task 1: Implementation


Construct and solve the model. For convenience, notice that we implement $F(q_i)$ as a single expression instead of $F(q_1)$ and $F(q_2)$ separately. $$F(q_i) = \beta q_i + \beta \sum_{j \in I} q_j - \alpha + \gamma_i.$$

In [ ]:
def duopoly_problem(gamma1, gamma2):

    # Initialize duopoly model (m)
    m = ConcreteModel()

    # Set of firms
    m.I = Set(initialize=[1, 2])

    # Define parameters
    m.gamma = Param(m.I, initialize={1:gamma1, 2:gamma2})

    # Define variables
    m.q = Var(m.I, domain=NonNegativeReals)

    # Complementarity constraints
    # 0 ≤ q_i ⊥ (gamma_i - a + b*sum(q_j) ) ≥ 0
    def constraint_q_rule(m, i):
        return complements(0 <= m.q[i], b * m.q[i] + b * sum(m.q[j] for j in m.I) - a + m.gamma[i] >= 0)
    m.constraint_q1 = Complementarity( m.I,rule=constraint_q_rule)

    ## Solve the model using PATH solver
    solver = SolverFactory('pathampl', executable=path_executable)
    results = solver.solve(m, tee=False) # tee=False suppresses solver output

    # Print termination condition
    print(f"\nSolver Status: {results.solver.termination_condition}")

    # Return model object with results
    return m

Function for displaying results

In [ ]:
def display_duopoly_results(m, gamma1, gamma2):
    # Display results
    print(f"\n{'='*60}")
    print(f"DUOPOLY EQUILIBRIUM RESULTS")
    print(f"{'='*60}")
    print(f"Marginal Costs: γ₁ = {gamma1}, γ₂ = {gamma2}")
    print(f"Demand: P = {a} - {b}*(q₁ + q₂)")
    print(f"\nEquilibrium Quantities:")

    q1_val = value(m.q[1])
    q2_val = value(m.q[2])
    Q_total = q1_val + q2_val
    P_market = a - b * Q_total

    print(f"  Firm 1: q₁ = {q1_val:.2f}")
    print(f"  Firm 2: q₂ = {q2_val:.2f}")
    print(f"  Total: Q = {Q_total:.2f}")
    print(f"\nMarket Price: P = {P_market:.2f}")

    # Calculate profits
    profit1 = (P_market - gamma1) * q1_val
    profit2 = (P_market - gamma2) * q2_val

    print(f"\nProfits:")
    print(f"  Firm 1: π₁ = {profit1:.2f}")
    print(f"  Firm 2: π₂ = {profit2:.2f}")
    print(f"  Total: Π = {profit1 + profit2:.2f}")
    print(f"{'='*60}\n")

Solve the model with given data

In [ ]:
duopoly_model = duopoly_problem(gamma1, gamma2)
display_duopoly_results(duopoly_model, gamma1, gamma2)

### Task 2: Sensitivity Analysis

Now suppose you are energy producer 1 and want to drive your competitor (producer 2) out of business. You can only control your costs $\gamma_1$ but could make them go negative (e.g., get subsidized by the government).

Do an analysis (numerical and/or analytical) to see the largest value of $\gamma_1$ that forces $q_2=0$ (put your competitor out of business).


In [ ]:
# Change these values to see how the results change
gamma1_new = 1

duopoly_new = duopoly_problem(gamma1_new, gamma2)
display_duopoly_results(duopoly_new, gamma1_new, gamma2)

### Task 3: Adding capacity constraints

With capacity constraint $q_i \leq q_i^{\max}$, producer 1's problem becomes:

\begin{align*}
\text{Maximize}_{q_1} \quad & (\alpha - \beta (q_1 + q_2)) q_1 - \gamma_1 q_1 \\
\text{s.t.} \quad & q_1 - q_1^{\max} \leq 0 \qquad (\delta_1) \\
& - q_1 \leq 0
\end{align*}

1) Derive the KKT conditions for this problem and reformulate the duopoly model as an MCP. Implement and solve the new model with $q_1^{\max} = 0.5$ and $q_2^{\max} = 2$.

2) Do the same sensitivity analysis as in Task 2 and find values for company 1 that force the competitor out of business.

In [ ]:
def duopoly_problem_with_capacity_constraints(gamma1, gamma2, q_max1, q_max2):

    # Initialize duopoly model (m)
    m = ConcreteModel()

    # Set of firms
    m.I = Set(initialize=[1, 2])

    # Define parameters
    m.gamma = Param(m.I, initialize={1:gamma1, 2:gamma2})
    m.q_max = Param(m.I, initialize={1:q_max1, 2:q_max2})

    # Define variables
    m.q = Var(m.I, domain=NonNegativeReals)
    m.delta = Var(m.I, domain=NonNegativeReals)

    # Complementarity constraints
    def constraint_q_rule(m, i):
        return complements(0 <= m.q[i], b * m.q[i] + b * sum(m.q[j] for j in m.I) - a + m.gamma[i] + m.delta[i] >= 0)
    m.constraint_q = Complementarity(m.I, rule=constraint_q_rule)

    def constraint_delta_rule(m, i):
        return complements(0 <= m.delta[i], m.q_max[i] - m.q[i] >= 0)
    m.constraint_delta = Complementarity(m.I, rule=constraint_delta_rule)

    # Solve the model using PATH solver
    solver = SolverFactory('pathampl', executable=path_executable)
    results = solver.solve(m, tee=False) # tee=False suppresses solver output

    # Print termination condition
    print(f"\nSolver Status: {results.solver.termination_condition}")

    # Return model object with results
    return m

Repeat the same analysis as above and find values for company 1 that force the competitor out of business. This time you can also control the capacity $q_1^{\max}$.

In [ ]:
# Change these values to see how the results change
gamma1_new = 1
q1_max = 0.5

# Do not change this value
q2_max = 1

duopoly_model_with_capacities = duopoly_problem_with_capacity_constraints(gamma1_new, gamma2, q1_max, q2_max)
display_duopoly_results(duopoly_model_with_capacities, gamma1, gamma2)

## Exercise 2.2: Multi-Node Energy Network Equilibrium

### Problem Description

We analyze a **two-node energy network** where four producers compete as price-takers in an asymmetric market structure. This creates a complex equilibrium with production, trading, and network flow decisions.

### Network Structure:
- **Node 1**: Producers A and B (can produce locally and export to Node 2)
- **Node 2**: Producers C and D (can only produce locally)
- **Network Operator**: Manages transmission flow $g_{12}$ between nodes
- **Asymmetric Market**: Only Node 1 producers can participate in inter-nodal trade

##### Define Problem Data and Parameters

In [ ]:
# Demand function parameters: D(π) = a - b*π
demand_params = {
    'node1': {'a': 20, 'b': 1},  # D₁(π₁) = 20 - π₁
    'node2': {'a': 40, 'b': 2}   # D₂(π₂) = 40 - 2*π₂
}

# Production cost parameters: γᵢ (marginal cost)
production_costs = {
    'A': 10,  # Producer A marginal cost
    'B': 12,  # Producer B marginal cost
    'C': 15,  # Producer C marginal cost
    'D': 18   # Producer D marginal cost
}

# Production capacity limits: q̄ᵢ
production_capacity = {
    'A': 10,    # Producer A capacity
    'B': 10,    # Producer B capacity
    'C': 5,     # Producer C capacity
    'D': 5      # Producer D capacity
}

# Transmission parameters
transmission_params = {
    'reg_tariff': 0.5,      # τ₁₂ᴿᵉᵍ: Regulated transmission tariff
    'tso_cost': 1,          # γᵀˢᴼ: TSO operational cost
    'capacity': 5,          # ḡ₁₂: Maximum transmission capacity
}


# Print problem parameters
print("=== MULTI-NODE ENERGY NETWORK DATA ===")

print("\nDemand Functions:")
for node, params in demand_params.items():
    print(f"  {node.title()}: D = {params['a']} - {params['b']}*π")

print("\nProduction Costs ($/MWh):")
for producer, cost in production_costs.items():
    node = 1 if producer in ['A', 'B'] else 2
    capacity = production_capacity[producer]
    print(f"  Producer {producer} (Node {node}): ${cost}/MWh, Capacity: {capacity} MW")

print(f"\nTransmission Parameters:")
print(f"  Regulated tariff: ${transmission_params['reg_tariff']}/MWh")
print(f"  TSO cost: ${transmission_params['tso_cost']}/MWh")
print(f"  Transmission capacity: {transmission_params['capacity']} MW")

### Task 1: Implementation

##### Mathematical Formulation

We will implement this model in Pyomo in parts and the formulation is shown in parts below. The formulation has

**Sets:**

{A, B} = Producers at Node 1

{C, D} = Producers at Node 2

{node1, node2} = Nodes in the network


**Parameters:**

$D_i(\pi_i)$ = demand function at node $i$

$\gamma_j$ = operational cost of producer $j$

$\bar{q}_i^j$ = production capacity limit for producer $j$ at node $i$  

$\tau_{12}^{Reg}$ = regulated transmission tariff

$\gamma^{TSO}$ = TSO operational cost  

$\bar{g}_{12}$ = maximum transmission capacity from node 1 to node 2  




In [ ]:
def define_sets_and_params(model, g_bar = 5):
    # Define sets
    model.Producers_Node1 = Set(initialize=['A', 'B'])
    model.Producers_Node2 = Set(initialize=['C', 'D'])
    model.Nodes = Set(initialize=['node1', 'node2'])

    # Define parameters
    # Demand function parameters: D(π) = a - b*π
    model.demand_a = Param(model.Nodes, initialize={'node1': 20, 'node2': 40})
    model.demand_b = Param(model.Nodes, initialize={'node1': 1, 'node2': 2})

    # Production cost and capacity
    model.gamma = Param(model.Producers_Node1 | model.Producers_Node2,
                    initialize={'A': 10, 'B': 12, 'C': 15, 'D': 18})
    model.q_bar = Param(model.Producers_Node1 | model.Producers_Node2,
                    initialize={'A': 10, 'B': 10, 'C': 5, 'D': 5})

    # Transmission parameters
    model.tau_reg = Param(initialize=0.5)  # Regulated transmission tariff
    model.gamma_tso = Param(initialize=1)  # TSO operational cost
    model.g_bar = Param(initialize=g_bar)  # Transmission capacity




**Decision Variables:**

$s_i^j$ = local sales by producer $j$ at node $i$  
$q_i^j$ = total production by producer $j$ at node $i$  
$f_{12}^j$ = inter-nodal flow from node 1 to node 2 by producer $j$  
$g_{12}$ = total transmission flow from node 1 to node 2    

**Dual variables:**

$\pi_i$ = energy price at node $i$  
$\tau_{12}$ = endogenous transmission tariff    
$\delta_i^j$ = shadow price of production balance constraint for producer $j$ at node $i$  
$\lambda_i^j$ = shadow price of capacity constraint for producer $j$ at node $i$  
$\varepsilon_{12}$ = shadow price of transmission capacity constraint   

In [ ]:
def add_variables(model):

    # Production and sales
    model.q = Var(model.Producers_Node1 | model.Producers_Node2, within=NonNegativeReals)           # Production
    model.s = Var(model.Producers_Node1 | model.Producers_Node2, within=NonNegativeReals)           # Local sales
    model.f = Var(model.Producers_Node1, within=NonNegativeReals)                                   # Inter-nodal flow (A, B only)
    model.lambda_var = Var(model.Producers_Node1 | model.Producers_Node2, within=NonNegativeReals)  # Capacity shadow price
    model.delta = Var(model.Producers_Node1 | model.Producers_Node2, within=Reals)                  # Production balance shadow price

    # Market variables
    model.pi = Var(model.Nodes, within=Reals)   # Nodal prices
    model.tau = Var(within=Reals)               # Endogenous transmission tariff

    # Network operator's decision variables
    model.g12 = Var(within=NonNegativeReals)  # Transmission flow
    model.eps = Var(within=NonNegativeReals)  # Transmission capacity shadow price

**Producer A (Node 1):**

$0 \leq -\pi_1 + \delta_1^A \perp s_1^A \geq 0$

$0 \leq \gamma_1^A + \lambda_1^A - \delta_1^A \perp q_1^A \geq 0$

$0 \leq -\pi_2 + (\tau_{12}^{Reg} + \tau_{12}) + \delta_1^A \perp f_{12}^A \geq 0$

$0 \leq \bar{q}_1^A - q_1^A \perp \lambda_1^A \geq 0$

$0 = s_1^A - q_1^A + f_{12}^A, \delta_1^A \text{ free}$

Note: these are *not* indexed constraints, thus we do not use a rule function to define them.

In [ ]:
def add_producer_a_constraints(m):
    # Producer A (Node 1)
    m.comp_s1A = Complementarity(expr =
        complements(0 <= m.s['A'], -m.pi['node1'] + m.delta['A'] >= 0))

    m.comp_q1A = Complementarity(expr=
        complements(0 <= m.q['A'], m.gamma['A'] + m.lambda_var['A'] - m.delta['A'] >= 0))

    m.comp_f12A = Complementarity(expr=
        complements(0 <= m.f['A'], -m.pi['node2'] + (m.tau_reg + m.tau) + m.delta['A'] >= 0))

    m.comp_lambda1A = Complementarity(expr=
        complements(0 <= m.q_bar['A'] - m.q['A'], m.lambda_var['A'] >= 0))

    m.balance_A = Constraint(expr = m.s['A'] - m.q['A'] + m.f['A'] == 0)

**Producer B (Node 1):**

$0 \leq -\pi_1 + \delta_1^B \perp s_1^B \geq 0$

$0 \leq \gamma_1^B + \lambda_1^B - \delta_1^B \perp q_1^B \geq 0$

$0 \leq -\pi_2 + (\tau_{12}^{Reg} + \tau_{12}) + \delta_1^B \perp f_{12}^B \geq 0$

$0 \leq \bar{q}_1^B - q_1^B \perp \lambda_1^B \geq 0$

$0 = s_1^B - q_1^B + f_{12}^B, \delta_1^B \text{ free}$


In [ ]:
def add_producer_b_constraints(m):
    # Producer B (Node 1)
    m.comp_s1B = Complementarity(expr =
        complements(0 <= m.s['B'], -m.pi['node1'] + m.delta['B'] >= 0))

    m.comp_q1B = Complementarity(expr =
        complements(0 <= m.q['B'], m.gamma['B'] + m.lambda_var['B'] - m.delta['B'] >= 0))

    m.comp_f12B = Complementarity(expr =
        complements(0 <= m.f['B'], -m.pi['node2'] + (m.tau_reg + m.tau) + m.delta['B'] >= 0))

    m.comp_lambda1B = Complementarity(expr =
        complements(0 <= m.q_bar['B'] - m.q['B'], m.lambda_var['B'] >= 0))

    m.balance_B = Constraint(expr = m.s['B'] - m.q['B'] + m.f['B'] == 0)

**Producer C (Node 2):**

$0 \leq -\pi_2 + \delta_2^C \perp s_2^C \geq 0$

$0 \leq \gamma_2^C + \lambda_2^C - \delta_2^C \perp q_2^C \geq 0$ (

$0 \leq \bar{q}_2^C - q_2^C \perp \lambda_2^C \geq 0$ (

$0 = s_2^C - q_2^C, \delta_2^C \text{ free}$

In [ ]:
def add_producer_c_constraints(m):
    # Producer C (Node 2)
    m.comp_s2C = Complementarity(expr =
        complements(0 <= m.s['C'], -m.pi['node2'] + m.delta['C'] >= 0))

    m.comp_q2C = Complementarity(expr =
        complements(0 <= m.q['C'], m.gamma['C'] + m.lambda_var['C'] - m.delta['C'] >= 0))

    m.comp_lambda2C = Complementarity(expr =
    complements(0 <= m.q_bar['C'] - m.q['C'], m.lambda_var['C'] >= 0))

    m.balance_C = Constraint(expr = m.s['C'] - m.q['C'] == 0)

**Producer D (Node 2):**

$0 \leq -\pi_2 + \delta_2^D \perp s_2^D \geq 0$

$0 \leq \gamma_2^D + \lambda_2^D - \delta_2^D \perp q_2^D \geq 0$

$0 \leq \bar{q}_2^D - q_2^D \perp \lambda_2^D \geq 0$

$0 = s_2^D - q_2^D, \delta_2^D \text{ free}$

In [ ]:
def add_producer_d_constraints(m):
    # Producer D (Node 2)
    m.comp_s2D = Complementarity(expr =
        complements(0 <= m.s['D'], -m.pi['node2'] + m.delta['D'] >= 0))

    m.comp_q2D = Complementarity(expr =
        complements(0 <= m.q['D'], m.gamma['D'] + m.lambda_var['D'] - m.delta['D'] >= 0))

    m.comp_lambda2D = Complementarity(expr =
        complements(0 <= m.q_bar['D'] - m.q['D'], m.lambda_var['D'] >= 0))

    m.balance_D = Constraint(expr = m.s['D'] - m.q['D'] == 0)

**Market Clearing Conditions:**

$0 = [s_1^A + s_1^B] - D_1(\pi_1), \pi_1 \text{ free}$

$0 = [s_2^C + s_2^D + f_{12}^A + f_{12}^B] - D_2(\pi_2), \pi_2 \text{ free}$

In [ ]:
def add_market_clearing_conditions(m):
    # Market clearing conditions
    m.market_clearing_node1 = Constraint(expr =
        m.s['A'] + m.s['B'] == m.demand_a['node1'] - m.demand_b['node1'] * m.pi['node1'])

    m.market_clearing_node2 = Constraint(expr =
        m.s['C'] + m.s['D'] + m.f['A'] + m.f['B'] == m.demand_a['node2'] - m.demand_b['node2'] * m.pi['node2'])

**Network Operator's Constraints:**

$0 \leq -\tau_{12}^{Reg} - \tau_{12} + \gamma^{TSO} + \varepsilon_{12} \perp g_{12} \geq 0$

$0 \leq \bar{g}_{12} - g_{12} \perp \varepsilon_{12} \geq 0$

$0 = g_{12} - [f_{12}^A + f_{12}^B], \tau_{12} \text{ free}$

In [ ]:
def add_network_operator_constraints(m):
    # Network operator's constraints
    m.comp_g12 = Complementarity(expr =
        complements(0 <= m.g12, -m.tau_reg - m.tau + m.gamma_tso + m.eps >= 0))

    m.comp_eps = Complementarity(expr =
        complements(0 <= m.g_bar - m.g12, m.eps >= 0))

    m.transmission_balance = Constraint(expr = m.g12 == m.f['A'] + m.f['B'])

Define a function that initializes the model and adds all components

In [ ]:
def create_model(gbar):
    # Create the Pyomo model
    model = ConcreteModel()
    define_sets_and_params(model, g_bar = gbar)
    add_variables(model)
    add_producer_a_constraints(model)
    add_producer_b_constraints(model)
    add_producer_c_constraints(model)
    add_producer_d_constraints(model)
    add_market_clearing_conditions(model)
    add_network_operator_constraints(model)
    return model

**Solve MCP Model**

In [ ]:
# Create the Pyomo model
model = create_model(gbar = transmission_params['capacity'])

# Solve the model
solver = SolverFactory('pathampl', executable=path_executable)
results = solver.solve(model)

In [ ]:
# Display results
print("=== MULTI-NODE ENERGY NETWORK RESULTS ===")
print(f"Solver Status: {results.solver.status}")
print(f"Termination Condition: {results.solver.termination_condition}")
print("\nProducer Results:")
for p in model.Producers_Node1 | model.Producers_Node2:
    node = 'node1' if p in model.Producers_Node1 else 'node2'
    print(f"\nProducer {p} (Node {node[-1]}):")
    print(f"  Production (q_{p}): {value(model.q[p]):.2f} MW")
    print(f"  Local Sales (s_{p}): {value(model.s[p]):.2f} MW")
    if p in model.Producers_Node1:
        print(f"  Export to Node 2 (f_{p}): {value(model.f[p]):.2f} MW")
    print(f"  Capacity Shadow Price (λ_{p}): {value(model.lambda_var[p]):.2f}")

print("\nMarket Results:")
print(f"  Node 1 Price (π₁): ${value(model.pi['node1']):.2f}/MWh")
print(f"  Node 2 Price (π₂): ${value(model.pi['node2']):.2f}/MWh")
print(f"  Transmission Flow (g₁₂): {value(model.g12):.2f} MW")
print(f"  Endogenous Tariff (τ): ${value(model.tau):.2f}/MWh")
print(f"  Transmission Capacity Shadow Price (ε): {value(model.eps):.2f}")


### Task 2

1. **Parameter Sensitivity**: How does the solution change if we change pipeline capacity in the TSO problem (3, 5, 10)? Do some sensitivity analysis.

2. **Transpetro**: Thought exercise: if we were to model the oil & gas context and Transpetro as the network operator, what are some other possible objectives Transpetro might have?

The following function takes the transmission capacity as input, solves the model, and displays the results.

In [ ]:
def solve_and_display_sensitivity_analysis(capacity):

    print(f"\n{'='*70}")
    print(f"CAPACITY = {capacity} MW")
    print(f"{'='*70}\n")

    # Solve
    model = create_model(gbar=capacity)
    solver = SolverFactory('pathampl', executable=path_executable)
    results = solver.solve(model, tee=False)

    # Quick display
    print(f"Status: {results.solver.termination_condition}")
    print(f"π₁ = ${value(model.pi['node1']):.2f}/MWh, π₂ = ${value(model.pi['node2']):.2f}/MWh")
    print(f"Flow = {value(model.g12):.2f} MW, Shadow Price (ε) = ${value(model.eps):.2f}/MWh\n")


Test different capacities and view results

In [ ]:
# Test different capacities - 3,5-baseline,10
for capacity in [3,5,10]:
    solve_and_display_sensitivity_analysis(capacity)